In [1]:
from pystac_client import Client
from odc.stac import load
import odc.geo

In [2]:
year = 2019
# client = Client.open("https://catalogue.dataspace.copernicus.eu/stac") 
client = Client.open("https://earth-search.aws.element84.com/v1") 
search = client.search(
    max_items=2,
    collections=['sentinel-2-l2a'],
    bbox=(-5.895070820833594, 43.311194627605936, -5.640165575300313, 43.49704758952146),
    datetime=f'{year}-04-01/{year}-10-01',
    query = ['eo:cloud_cover<50']
)
print(f"{search.matched()} items found")
item_collection = search.item_collection()

61 items found


In [8]:
bands_sentinel2 = ['blue','red','nir','swir16','swir22']
xx = odc.stac.load(
    item_collection,
    bands= bands_sentinel2,
    crs="EPSG:25830",
    resolution=50
)

In [8]:
xx

<xarray.Dataset> Size: 6GB
Dimensions:      (y: 11763, x: 11762, time: 10)
Coordinates:
  * y            (y) float64 94kB 4.818e+06 4.818e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 94kB 5.835e+03 5.845e+03 ... 1.234e+05 1.234e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 80B 2022-12-08T11:39:28.168000 ... 202...
Data variables:
    blue         (time, y, x) uint16 3GB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    swir16       (time, y, x) uint16 3GB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [9]:

xx.median(dim='time',skipna=True)

<xarray.Dataset> Size: 2GB
Dimensions:      (y: 11763, x: 11762)
Coordinates:
  * y            (y) float64 94kB 4.818e+06 4.818e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 94kB 5.835e+03 5.845e+03 ... 1.234e+05 1.234e+05
    spatial_ref  int32 4B 25830
Data variables:
    blue         (y, x) float64 1GB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    swir16       (y, x) float64 1GB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [76]:
xx.spatial_ref.GeoTransform


'5600 400 0 4817600 0 -400'

In [62]:
ndvi = (xx.red - xx.nir) / (xx.red + xx.nir)

ndvi_p10 = ndvi.quantile(0.1,dim='time',skipna = True)
ndvi_p90 = ndvi.quantile(0.9,dim='time',skipna = True)
ndvi_median = ndvi.median(dim='time', skipna = True)

ndvi_median.fillna(255)

/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


<xarray.DataArray (y: 295, x: 295)> Size: 696kB
array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]])
Coordinates:
  * y            (y) float64 2kB 4.817e+06 4.817e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 2kB 5.8e+03 6.2e+03 6.6e+03 ... 1.23e+05 1.234e+05
    spatial_ref  int32 4B 25830

In [72]:
import xarray as xr
import rioxarray as rio

# median_blue = xx.blue.median(dim='time',skipna=True)
# median_red = xx.red.median(dim='time',skipna=True)
# median_green = xx.green.median(dim='time',skipna=True)
# median_nir = xx.nir.median(dim='time',skipna=True)
# median_swir16 = xx.swir16.median(dim='time',skipna=True)
# median_swir22 = xx.swir22.median(dim='time',skipna=True)

# median_blue

# xx

xx.median(dim='time',skipna=True)


# dataset = xr.merge([median_blue,median_red,median_green])
# dataset.rio.to_raster("test_multiband.tif")

<xarray.Dataset> Size: 4MB
Dimensions:      (y: 295, x: 295)
Coordinates:
  * y            (y) float64 2kB 4.817e+06 4.817e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 2kB 5.8e+03 6.2e+03 6.6e+03 ... 1.23e+05 1.234e+05
    spatial_ref  int32 4B 25830
Data variables:
    blue         (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    green        (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    red          (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    nir          (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    swir16       (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    swir22       (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0